In [ ]:
## to run in colab

# !git clone https://github.com/sergeychuvakin/advanced_nlp_course.git
# !mv advanced_nlp_course/LM/*.py ./
# !mv advanced_nlp_course/LM/*.json ./ ## 
# !pip install loguru pydantic tokenizers

In [132]:
!pip freeze | egrep "pydantic|torch|loguru|tokenizers|requests|nltk|tqdm"

loguru==0.5.3
nltk==3.6.2
pydantic==1.8.2
requests==2.25.1
requests-oauthlib==1.3.0
tokenizers==0.10.3
torch==1.9.0
torchtext==0.6.0
tqdm==4.59.0


In [135]:
%reload_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
from loguru import logger
import sys
import json


from dependencies import corpus, tokenizer
from config import Config, LanguageModelConfig
from processing_utils import (
    clean_text, 
    split_on_sequences, 
    create_ngrams, 
    create_to_x_and_y, 
    word2int,
    create_vocab,
    save_artifacts
)
from model import LM_LSTM
from datahandler import LMDataset
from train_utils import train_model

config = Config()

logger.remove()
logger.add(sys.stderr, level="WARNING")

19

In [ ]:
corpus = clean_text(corpus)
corpus = split_on_sequences(corpus)

tcorpus = tokenizer.encode_batch(corpus)

## create n-grams for each doc
sq = create_ngrams(tcorpus, config.N_GRAM) 
 
## shift corpus to create x and y 
x, y =  create_to_x_and_y(sq)

id_token, token_id = create_vocab(tokenizer)
vocab_size = len(token_id)

## split data
tradeoff_index = int(len(x) * config.TRAIN_PROPORTION)

x_train = x[:tradeoff_index]
x_test = x[tradeoff_index:]

y_train = y[:tradeoff_index]
y_test = y[tradeoff_index:]

logger.warning(
    f"""
    Output shapes: 
        x_train: {len(x_train)}, 
        x_test: {len(x_test)}, 
        y_train: {len(y_train)}, 
        y_test: {len(y_test)}
    """
              )

## load to dataset and dataloader
train_ds = LMDataset(x_train, y_train)
test_ds = LMDataset(x_test, y_test)

train_dl = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=config.BATCH_SIZE, shuffle=False)

# model and model config
model_config = LanguageModelConfig(vocab_size=vocab_size, emb_size=300)
model = LM_LSTM(**model_config.dict(), logger=logger)

## save artifacts
save_artifacts(
    (model_config.dict(), config.SAVE_MODEL_CONFIG),
    (token_id, config.SAVE_TOKEN_ID),
    (id_token, config.SAVE_ID_TOKEN)
)

optimizer = torch.optim.Adam(model.parameters(), lr=model_config.lr)
loss_func = torch.nn.CrossEntropyLoss()


# train model 
tmodel = train_model(model,
                     train_dl,
                     optimizer=optimizer,
                     loss_func=loss_func,
                     epochs=5, 
                     clip=1)

torch.save(model.state_dict(), config.SAVE_MODEL_FNAME)

### Validation

In [159]:
from validation_utils import val_metrics

logger.warning(
    """
    Cross-Entropy: %f
    Perpelxity: %f
    """ % 
    val_metrics(model, test_dl, token_id, loss_func)
)

2021-11-12 16:01:24.897 | WARNING  | __main__:<module>:8 - 
    Cross-Entropy: 10.255110
    Perpelxity: 28427.429688
    


### Inference

In [7]:
from inference import (
    model, 
    token_id, 
    id_token, 
    model, 
    predict_one_word, 
    predict_sample
)

from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, level="WARNING")

1

In [8]:
predict_one_word(
    "one of the",  
    model, 
    token_id, 
    id_token,
    random=False
)

','

In [9]:
predict_sample(
    "language modeling",  
    model, 
    token_id, 
    id_token,
    length_of_sample=10,
    random=True
)

'the , the . the the . , . the'